In [6]:
import pandas as pd
import numpy as np

# Carregar dados
df = pd.read_csv("database/database_transformed.csv")

# Ligas para remover
ligas_remover = [
    "PRMP",
    "PCS",
    "HC",
    "LAS",
    "VCS",
    "LJL",
    "EBL",
    "LPLOL",
    "CT",
    "AL",
    "HW",
    "HLL",
    "Asia Master",
    "RL",
    "LRN",
    "NEXO",
    "LFL2",
    "ROL",
    "LTA",
]

# Ligas Tier 1
ligas_tier1 = ["LPL", "LCK", "LEC", "LCS"]


def analisar_impacto_campeoes(df, nome_dataset="Dataset"):
    # Extrair todos os campeões de todas as posições
    champions_data = []
    positions = [
        "top_t1",
        "jung_t1",
        "mid_t1",
        "adc_t1",
        "sup_t1",
        "top_t2",
        "jung_t2",
        "mid_t2",
        "adc_t2",
        "sup_t2",
    ]

    for _, row in df.iterrows():
        for pos in positions:
            if pd.notna(row[pos]) and pd.notna(row["total_kills"]):
                champions_data.append(
                    {
                        "champion": row[pos],
                        "total_kills": row["total_kills"],
                        "position": pos.split("_")[0],
                    }
                )

    champions_df = pd.DataFrame(champions_data)

    # Calcular estatísticas por campeão
    champion_stats = (
        champions_df.groupby("champion")
        .agg({"total_kills": ["mean", "count", "std"]})
        .round(2)
    )

    champion_stats.columns = ["avg_kills", "games", "std_kills"]
    champion_stats = champion_stats.reset_index()

    # Filtrar campeões com pelo menos 75 jogos
    champion_stats = champion_stats[champion_stats["games"] >= 75].copy()

    # Calcular média geral
    media_geral = df["total_kills"].mean()

    # Calcular impacto (diferença da média)
    champion_stats["impacto"] = champion_stats["avg_kills"] - media_geral
    champion_stats["impacto_pct"] = (
        champion_stats["impacto"] / media_geral * 100
    ).round(2)

    # Ordenar por impacto
    champion_stats = champion_stats.sort_values("impacto", ascending=False)

    print(f"\n=== ANÁLISE {nome_dataset.upper()} ===")
    print(f"Total de jogos: {len(df):,}")
    print(f"Média geral de kills: {media_geral:.2f}")
    print(f"Campeões analisados (≥75 jogos): {len(champion_stats)}")

    print(f"\n🔥 TOP 20 CAMPEÕES - MAIOR IMPACTO (+)")
    top_20 = champion_stats.head(20)
    for i, row in top_20.iterrows():
        print(
            f"{i + 1:2d}. {row['champion']:12} | {row['avg_kills']:5.1f} kills | +{row['impacto']:4.1f} | {row['impacto_pct']:+5.1f}% | {row['games']} jogos"
        )

    print(f"\n❄️  TOP 20 CAMPEÕES - MENOR IMPACTO (-)")
    bottom_20 = champion_stats.tail(20)
    for i, row in bottom_20.iterrows():
        idx = len(champion_stats) - 20 + (i - bottom_20.index[0])
        print(
            f"{idx + 1:2d}. {row['champion']:12} | {row['avg_kills']:5.1f} kills | {row['impacto']:+4.1f} | {row['impacto_pct']:+5.1f}% | {row['games']} jogos"
        )

    return champion_stats


def analisar_annie(df, nome_dataset):
    positions = [
        "top_t1",
        "jung_t1",
        "mid_t1",
        "adc_t1",
        "sup_t1",
        "top_t2",
        "jung_t2",
        "mid_t2",
        "adc_t2",
        "sup_t2",
    ]

    annie_games = []
    for _, row in df.iterrows():
        for pos in positions:
            if row[pos] == "Annie":
                annie_games.append(
                    {
                        "total_kills": row["total_kills"],
                        "position": pos.split("_")[0],
                        "team": pos.split("_")[1],
                        "league": row["league"],
                    }
                )

    if not annie_games:
        print(f"\n🔍 ANNIE - {nome_dataset}: Não encontrada nos dados")
        return

    annie_df = pd.DataFrame(annie_games)
    media_geral = df["total_kills"].mean()

    print(f"\n🔍 ANÁLISE DETALHADA - ANNIE ({nome_dataset})")
    print(f"Jogos com Annie: {len(annie_df)}")
    print(f"Média de kills com Annie: {annie_df['total_kills'].mean():.2f}")
    print(f"Média geral: {media_geral:.2f}")
    print(f"Impacto: {annie_df['total_kills'].mean() - media_geral:+.2f} kills")
    impacto_pct = (annie_df["total_kills"].mean() - media_geral) / media_geral * 100
    print(f"Impacto percentual: {impacto_pct:+.1f}%")

    if len(annie_df) > 1:
        print(f"Desvio padrão: {annie_df['total_kills'].std():.2f}")


def comparar_datasets(stats_sem_outliers, stats_tier1):
    # Merge dos datasets para comparação
    comparacao = pd.merge(
        stats_sem_outliers[
            ["champion", "avg_kills", "impacto", "impacto_pct", "games"]
        ],
        stats_tier1[["champion", "avg_kills", "impacto", "impacto_pct", "games"]],
        on="champion",
        suffixes=("_sem_outliers", "_tier1"),
        how="outer",
    )

    # Filtrar apenas campeões que aparecem em ambos datasets
    comparacao = comparacao.dropna()

    # Calcular diferença de impacto entre os datasets
    comparacao["diff_impacto"] = (
        comparacao["impacto_tier1"] - comparacao["impacto_sem_outliers"]
    )
    comparacao["diff_impacto_pct"] = (
        comparacao["impacto_pct_tier1"] - comparacao["impacto_pct_sem_outliers"]
    )

    # Ordenar por maior diferença absoluta
    comparacao["diff_abs"] = abs(comparacao["diff_impacto"])
    comparacao = comparacao.sort_values("diff_abs", ascending=False)

    print("\n" + "=" * 70)
    print("🔄 COMPARAÇÃO: SEM OUTLIERS vs TIER 1")
    print("=" * 70)
    print(f"Campeões em ambos datasets: {len(comparacao)}")

    print("\n📈 TOP 20 - MAIORES DIFERENÇAS DE IMPACTO")
    print("(Positivo = Mais impacto em Tier 1 | Negativo = Menos impacto em Tier 1)")

    top_diff = comparacao.head(20)
    for i, row in top_diff.iterrows():
        sinal = "📈" if row["diff_impacto"] > 0 else "📉"
        print(
            f"{sinal} {row['champion']:12} | Sem Out: {row['impacto_sem_outliers']:+4.1f} | Tier1: {row['impacto_tier1']:+4.1f} | Diff: {row['diff_impacto']:+4.1f}"
        )


# ANÁLISE 1: Sem ligas outliers
print("=" * 60)
print("ANÁLISE FILTRADA - SEM LIGAS OUTLIERS")
print("=" * 60)
df_filtrado = df[~df["league"].isin(ligas_remover)].copy()
stats_filtradas = analisar_impacto_campeoes(df_filtrado, "SEM OUTLIERS")

# ANÁLISE 2: Apenas ligas Tier 1
print("\n" + "=" * 60)
print("ANÁLISE LIGAS TIER 1 - LPL, LCK, LEC, LCS")
print("=" * 60)
df_tier1 = df[df["league"].isin(ligas_tier1)].copy()
stats_tier1 = analisar_impacto_campeoes(df_tier1, "TIER 1")

# COMPARAÇÃO entre datasets
comparar_datasets(stats_filtradas, stats_tier1)

# Análise Annie
analisar_annie(df_filtrado, "SEM OUTLIERS")
analisar_annie(df_tier1, "TIER 1")

print("\n" + "=" * 60)
print("RESUMO EXECUTIVO")
print("=" * 60)
print("✅ Análise sem outliers vs Tier 1 realizada")
print("📊 Comparação de impactos entre datasets")
print("🎯 Foco em campeões com ≥75 jogos")
print("🔄 Identificadas mudanças relevantes entre cenários")

ANÁLISE FILTRADA - SEM LIGAS OUTLIERS

=== ANÁLISE SEM OUTLIERS ===
Total de jogos: 5,539
Média geral de kills: 30.31
Campeões analisados (≥75 jogos): 90

🔥 TOP 20 CAMPEÕES - MAIOR IMPACTO (+)
99. Pyke         |  33.7 kills | + 3.4 | +11.3% | 99 jogos
164. Zed          |  33.1 kills | + 2.8 |  +9.3% | 109 jogos
88. Nautilus     |  32.0 kills | + 1.7 |  +5.6% | 1194 jogos
97. Pantheon     |  31.9 kills | + 1.6 |  +5.3% | 890 jogos
 3. Akali        |  31.9 kills | + 1.6 |  +5.3% | 384 jogos
50. Jarvan IV    |  31.9 kills | + 1.6 |  +5.3% | 486 jogos
118. Shen         |  31.9 kills | + 1.6 |  +5.2% | 141 jogos
71. Leona        |  31.6 kills | + 1.2 |  +4.1% | 1065 jogos
31. Elise        |  31.4 kills | + 1.1 |  +3.5% | 128 jogos
131. Taliyah      |  31.2 kills | + 0.9 |  +3.1% | 1272 jogos
110. Rumble       |  31.2 kills | + 0.9 |  +2.9% | 1367 jogos
129. Syndra       |  31.1 kills | + 0.8 |  +2.5% | 225 jogos
149. Viego        |  31.0 kills | + 0.7 |  +2.3% | 402 jogos
17. Blitzcrank   |

In [10]:
import pandas as pd
import numpy as np

# Carregar dados
df = pd.read_csv("database/database_transformed.csv")

# Ligas Tier 1
ligas_tier1 = ["LPL", "LCK", "LEC", "LCS"]


def analisar_impacto_campeoes(df, nome_dataset="Dataset"):
    # Extrair todos os campeões de todas as posições
    champions_data = []
    positions = [
        "top_t1",
        "jung_t1",
        "mid_t1",
        "adc_t1",
        "sup_t1",
        "top_t2",
        "jung_t2",
        "mid_t2",
        "adc_t2",
        "sup_t2",
    ]

    for _, row in df.iterrows():
        for pos in positions:
            if pd.notna(row[pos]) and pd.notna(row["total_kills"]):
                champions_data.append(
                    {
                        "champion": row[pos],
                        "total_kills": row["total_kills"],
                        "position": pos.split("_")[0],
                    }
                )

    champions_df = pd.DataFrame(champions_data)

    # Calcular estatísticas por campeão
    champion_stats = (
        champions_df.groupby("champion")
        .agg({"total_kills": ["mean", "count", "std"]})
        .round(2)
    )

    champion_stats.columns = ["avg_kills", "games", "std_kills"]
    champion_stats = champion_stats.reset_index()

    # Filtrar campeões com pelo menos 75 jogos para Tier 1 geral, 15 para individuais
    min_games = 75 if "TIER 1" == nome_dataset else 15
    champion_stats = champion_stats[champion_stats["games"] >= min_games].copy()

    # Calcular média geral
    media_geral = df["total_kills"].mean()

    # Calcular impacto (diferença da média)
    champion_stats["impacto"] = champion_stats["avg_kills"] - media_geral
    champion_stats["impacto_pct"] = (
        champion_stats["impacto"] / media_geral * 100
    ).round(2)

    # Ordenar por impacto
    champion_stats = champion_stats.sort_values("impacto", ascending=False)

    print(f"\n=== ANÁLISE {nome_dataset.upper()} ===")
    print(f"Total de jogos: {len(df):,}")
    print(f"Média geral de kills: {media_geral:.2f}")
    print(f"Campeões analisados (≥{min_games} jogos): {len(champion_stats)}")

    print(f"\n🔥 TOP 20 CAMPEÕES - MAIOR IMPACTO (+)")
    top_20 = champion_stats.head(20)
    for i, row in top_20.iterrows():
        print(
            f"{i + 1:2d}. {row['champion']:12} | {row['avg_kills']:5.1f} kills | +{row['impacto']:4.1f} | {row['impacto_pct']:+5.1f}% | {row['games']} jogos"
        )

    print(f"\n❄️  TOP 20 CAMPEÕES - MENOR IMPACTO (-)")
    bottom_20 = champion_stats.tail(20)
    for i, row in bottom_20.iterrows():
        idx = len(champion_stats) - 20 + (i - bottom_20.index[0])
        print(
            f"{idx + 1:2d}. {row['champion']:12} | {row['avg_kills']:5.1f} kills | {row['impacto']:+4.1f} | {row['impacto_pct']:+5.1f}% | {row['games']} jogos"
        )

    return champion_stats


def analisar_liga_individual(df, liga, media_tier1_geral):
    df_liga = df[df["league"] == liga].copy()

    if len(df_liga) == 0:
        print(f"\n⚠️ {liga}: Não há dados")
        return None

    # Extrair campeões da liga específica
    champions_data = []
    positions = [
        "top_t1",
        "jung_t1",
        "mid_t1",
        "adc_t1",
        "sup_t1",
        "top_t2",
        "jung_t2",
        "mid_t2",
        "adc_t2",
        "sup_t2",
    ]

    for _, row in df_liga.iterrows():
        for pos in positions:
            if pd.notna(row[pos]) and pd.notna(row["total_kills"]):
                champions_data.append(
                    {"champion": row[pos], "total_kills": row["total_kills"]}
                )

    champions_df = pd.DataFrame(champions_data)

    # Calcular estatísticas por campeão (mínimo 15 jogos)
    champion_stats = (
        champions_df.groupby("champion")
        .agg({"total_kills": ["mean", "count", "std"]})
        .round(2)
    )

    champion_stats.columns = ["avg_kills", "games", "std_kills"]
    champion_stats = champion_stats.reset_index()
    champion_stats = champion_stats[champion_stats["games"] >= 15].copy()

    # Média da liga
    media_liga = df_liga["total_kills"].mean()

    # Calcular impacto vs média da liga
    champion_stats["impacto"] = champion_stats["avg_kills"] - media_liga
    champion_stats["impacto_pct"] = (
        champion_stats["impacto"] / media_liga * 100
    ).round(2)

    # Ordenar por impacto
    champion_stats = champion_stats.sort_values("impacto", ascending=False)

    print(f"\n🏆 {liga} - ANÁLISE INDIVIDUAL")
    print(
        f"Jogos: {len(df_liga):,} | Média: {media_liga:.2f} kills | Campeões: {len(champion_stats)} | Diff vs Tier1: {media_liga - media_tier1_geral:+.2f}"
    )

    # Top 15 maior impacto
    print(f"\n🔥 TOP 15 - MAIOR IMPACTO EM {liga}")
    top_15 = champion_stats.head(15)
    for i, row in top_15.iterrows():
        print(
            f"{i + 1:2d}. {row['champion']:12} | {row['avg_kills']:5.1f} kills | +{row['impacto']:4.1f} | {row['impacto_pct']:+5.1f}% | {row['games']} jogos"
        )

    # Top 15 menor impacto
    print(f"\n❄️  TOP 15 - MENOR IMPACTO EM {liga}")
    bottom_15 = champion_stats.tail(15)
    for i, row in bottom_15.iterrows():
        idx = len(champion_stats) - 15 + (i - bottom_15.index[0])
        print(
            f"{idx + 1:2d}. {row['champion']:12} | {row['avg_kills']:5.1f} kills | {row['impacto']:+4.1f} | {row['impacto_pct']:+5.1f}% | {row['games']} jogos"
        )

    return champion_stats


# ANÁLISE TIER 1 GERAL
print("=" * 60)
print("ANÁLISE LIGAS TIER 1 GERAL - LPL, LCK, LEC, LCS")
print("=" * 60)
df_tier1 = df[df["league"].isin(ligas_tier1)].copy()
stats_tier1 = analisar_impacto_campeoes(df_tier1, "TIER 1")

# Salvar média Tier 1 para comparações
media_tier1_geral = df_tier1["total_kills"].mean()

# ANÁLISE INDIVIDUAL DE CADA LIGA
print("\n" + "=" * 70)
print("🏆 ANÁLISE INDIVIDUAL POR LIGA - TIER 1")
print("=" * 70)

for liga in ["LPL", "LCK", "LEC", "LCS"]:
    analisar_liga_individual(df_tier1, liga, media_tier1_geral)

# RESUMO COMPARATIVO DAS LIGAS
print(f"\n" + "=" * 50)
print("📊 RESUMO COMPARATIVO DAS LIGAS TIER 1")
print("=" * 50)

# Estatísticas básicas das ligas
for liga in ["LPL", "LCK", "LEC", "LCS"]:
    df_liga = df_tier1[df_tier1["league"] == liga]
    if len(df_liga) > 0:
        media = df_liga["total_kills"].mean()
        diff_tier1 = media - media_tier1_geral
        print(
            f"  {liga:3} | {len(df_liga):4} jogos | {media:5.2f} kills | {diff_tier1:+4.1f} vs Média Tier1"
        )

# Comparação de campeões entre ligas
print(f"\n🔥 COMPARAÇÃO DE CAMPEÕES - TOP IMPACTOS POR LIGA")

# Coletar dados de todas as ligas individuais
liga_stats = {}
for liga in ["LPL", "LCK", "LEC", "LCS"]:
    df_liga = df_tier1[df_tier1["league"] == liga].copy()

    if len(df_liga) > 0:
        # Extrair campeões
        champions_data = []
        positions = [
            "top_t1",
            "jung_t1",
            "mid_t1",
            "adc_t1",
            "sup_t1",
            "top_t2",
            "jung_t2",
            "mid_t2",
            "adc_t2",
            "sup_t2",
        ]

        for _, row in df_liga.iterrows():
            for pos in positions:
                if pd.notna(row[pos]) and pd.notna(row["total_kills"]):
                    champions_data.append(
                        {"champion": row[pos], "total_kills": row["total_kills"]}
                    )

        champions_df = pd.DataFrame(champions_data)

        # Calcular stats (mín 10 jogos para comparação)
        champion_stats = (
            champions_df.groupby("champion")
            .agg({"total_kills": ["mean", "count"]})
            .round(2)
        )

        champion_stats.columns = ["avg_kills", "games"]
        champion_stats = champion_stats.reset_index()
        champion_stats = champion_stats[champion_stats["games"] >= 10].copy()

        # Calcular impacto vs média da liga
        media_liga = df_liga["total_kills"].mean()
        champion_stats["impacto"] = champion_stats["avg_kills"] - media_liga

        liga_stats[liga] = champion_stats.sort_values("impacto", ascending=False)

# Encontrar campeões que aparecem em múltiplas ligas
all_champions = set()
for liga_data in liga_stats.values():
    all_champions.update(liga_data["champion"].tolist())

# Comparar campeões entre ligas
print(f"\n🔍 CAMPEÕES COM MAIOR VARIAÇÃO ENTRE LIGAS (≥10 jogos por liga)")
comparacoes = []

for champion in all_champions:
    liga_impacts = {}
    liga_games = {}

    for liga, data in liga_stats.items():
        champ_data = data[data["champion"] == champion]
        if len(champ_data) > 0:
            liga_impacts[liga] = champ_data["impacto"].iloc[0]
            liga_games[liga] = champ_data["games"].iloc[0]

    # Só comparar se aparecer em pelo menos 3 ligas
    if len(liga_impacts) >= 3:
        max_impact = max(liga_impacts.values())
        min_impact = min(liga_impacts.values())
        variacao = max_impact - min_impact

        if variacao >= 1.0:  # Só mostrar variações relevantes
            comparacoes.append(
                {
                    "champion": champion,
                    "variacao": variacao,
                    "max_liga": max(liga_impacts, key=liga_impacts.get),
                    "min_liga": min(liga_impacts, key=liga_impacts.get),
                    "max_impact": max_impact,
                    "min_impact": min_impact,
                    "ligas": liga_impacts,
                    "jogos": liga_games,
                }
            )

# Ordenar por maior variação
comparacoes = sorted(comparacoes, key=lambda x: x["variacao"], reverse=True)

for comp in comparacoes[:15]:  # Top 15 maiores variações
    print(f"\n📊 {comp['champion']:12} | Variação: {comp['variacao']:4.1f}")
    for liga in ["LPL", "LCK", "LEC", "LCS"]:
        if liga in comp["ligas"]:
            impacto = comp["ligas"][liga]
            jogos = comp["jogos"][liga]
            destaque = (
                "🔥"
                if liga == comp["max_liga"]
                else "❄️"
                if liga == comp["min_liga"]
                else "  "
            )
            print(f"    {destaque} {liga}: {impacto:+4.1f} ({jogos} jogos)")

print(f"\n🏆 TOP 5 CAMPEÕES DOMINANTES POR LIGA")
for liga in ["LPL", "LCK", "LEC", "LCS"]:
    if liga in liga_stats:
        top_5 = liga_stats[liga].head(5)
        print(f"\n{liga}:")
        for _, row in top_5.iterrows():
            print(
                f"  {row['champion']:12} +{row['impacto']:4.1f} ({row['games']} jogos)"
            )

print(f"\n❄️  BOTTOM 3 CAMPEÕES POR LIGA")
for liga in ["LPL", "LCK", "LEC", "LCS"]:
    if liga in liga_stats:
        bottom_3 = liga_stats[liga].tail(3)
        print(f"\n{liga}:")
        for _, row in bottom_3.iterrows():
            print(
                f"  {row['champion']:12} {row['impacto']:+4.1f} ({row['games']} jogos)"
            )

print("\n" + "=" * 60)
print("RESUMO EXECUTIVO")
print("=" * 60)
print("✅ Análise Tier 1 geral + análise individual por liga")
print("📊 Comparação de cada liga vs média Tier 1")
print("🎯 Tier 1 geral: ≥75 jogos | Por liga: ≥15 jogos")
print("🏆 Identificação de diferenças regionais")

ANÁLISE LIGAS TIER 1 GERAL - LPL, LCK, LEC, LCS

=== ANÁLISE TIER 1 ===
Total de jogos: 1,633
Média geral de kills: 29.13
Campeões analisados (≥75 jogos): 64

🔥 TOP 20 CAMPEÕES - MAIOR IMPACTO (+)
 3. Akali        |  31.4 kills | + 2.3 |  +7.8% | 115 jogos
78. Nautilus     |  31.0 kills | + 1.9 |  +6.5% | 336 jogos
86. Pantheon     |  30.8 kills | + 1.7 |  +5.8% | 298 jogos
61. Leona        |  30.6 kills | + 1.5 |  +5.2% | 296 jogos
79. Neeko        |  30.4 kills | + 1.3 |  +4.5% | 234 jogos
 8. Annie        |  30.4 kills | + 1.3 |  +4.4% | 173 jogos
82. Nocturne     |  30.3 kills | + 1.1 |  +3.9% | 109 jogos
46. Jhin         |  30.2 kills | + 1.1 |  +3.8% | 277 jogos
115. Taliyah      |  30.2 kills | + 1.1 |  +3.7% | 411 jogos
64. Lucian       |  30.2 kills | + 1.0 |  +3.6% | 177 jogos
49. Kai'Sa       |  30.1 kills | + 0.9 |  +3.3% | 339 jogos
97. Rumble       |  30.1 kills | + 0.9 |  +3.2% | 433 jogos
43. Jarvan IV    |  30.0 kills | + 0.9 |  +3.1% | 131 jogos
139. Xin Zhao     |  3